# Pivot Tables and Cross-Tabulation
*Curtis Miller*

In this notebook I demonstrate the use of pivot tables and cross-tabulation.

In [ ]:
import pandas as pd
import numpy as np

## Datasets

I consider two datasets in this notebook. The first dataset is a subset of the [American National Election Studies (ANES)](http://www.electionstudies.org/) survey dataset, including potential voters' education level, age group, voter registration status, and who they (self-reported) voted for in the 2016 Presidential election. Given the small size of this subset of the dataset (the actual dataset is much larger and available for free, though loading it into Python may be tricky), I do some post-processing, using broader groups.

In [ ]:
voters = pd.read_csv("VoterData.csv")

# Some data processing, merging groups together
voters.Education.replace({"Less than high school": "No Bachelors",
                          "High school": "No Bachelors",
                          "Associate": "No Bachelors",
                          "Bachelors": "Bachelors or more",
                          "Graduate": "Bachelors or more"}, inplace=True)
voters.AgeGroup.replace({"[18, 30)": "[18, 50)",
                         "[30, 40)": "[18, 50)",
                         "[40, 50)": "[18, 50)",
                         "[50, 60)": "[50, 90)",
                         "[60, 70)": "[50, 90)",
                         "[70, 80)": "[50, 90)",
                         "[80, 90)": "[50, 90)"}, inplace=True)

# Data should be categorical for our purposes (plus, this is correct format)
voters.AgeGroup = pd.Categorical(voters.AgeGroup)
voters.Education = pd.Categorical(voters.Education)
voters.VotedFor = pd.Categorical(voters.VotedFor)

voters.head()

In [ ]:
voters.dtypes

The next dataset is the familiar population pyramid dataset, but restricted to just the United States.

In [ ]:
pop_pyramids = pd.read_csv("PopPyramids.csv")
# Restrict to just USA, and don't include "Total" rows; plus, include only male and female population counts
ppUSA = pop_pyramids.loc[(pop_pyramids.Country == "UnitedStates") & (pop_pyramids.Age != "Total"),
                         ["Age", "Year", "Male Population", "Female Population"]]
ppUSA.columns = pd.Index(["Age", "Year", "Male", "Female"])
ppUSA.head()

In [ ]:
# Turn to long-form format; must be done for good pivot tables
ppUSA = pd.melt(ppUSA, id_vars=["Age", "Year"], var_name="Sex", value_name="Population")
ppUSA.head()

## Cross-Tabulation

In cross-tabulation, we pass `Series` to the pandas function `crosstab()`. These can be lists of multiple `Series`. The result is a table that counts the frequency of occurance of combinations of possible values in the `Series`.

In [ ]:
pd.crosstab(voters.Education, voters.VotedFor)    # See how many people with different educations voted for whom

In [ ]:
pd.crosstab(voters.AgeGroup, voters.VotedFor, margins=True)    # Do the same for age groups; also, include margin totals

In [ ]:
pd.crosstab(voters.AgeGroup, voters.Education)    # What is the joint frequency of age and education level?

In [ ]:
# If Registered == 1, the individual is registered to vote
# This table counts how frequently combinations of age and educcation are to be registered/not registered to vote
pd.crosstab([voters.AgeGroup, voters.Education], voters.Registered)

In [ ]:
pd.crosstab([voters.AgeGroup, voters.Education], voters.VotedFor, margins=True)

## Pivot Tables

Pivot tables are effectively another form of cross-tabulation. We construct a table where each cell is a statistical summary of a dataset, entries mapped to a cell being aggregated together according to some aggregation function. We choose which variables should be row variables, which should be column variables, and what variable constitutes "values" that the table aggregates.

There is a pandas function, `pivot_table()`, that constructs pivot tables. If data is in a `DataFrame`, though, the `pivot_table()` method is easier and faster to use.

### Population Data
Here I construct pivot tables for the population data, thus getting total population counts. The only reasonable way to aggregate is by summing populations together. Notice that the values are always `Population`.

In [ ]:
ppUSA.pivot_table(index="Age", columns="Year", values="Population", aggfunc="sum")  # Populations in age groups over time

In [ ]:
ppUSA.pivot_table(index="Sex", columns="Year", values="Population", aggfunc="sum")    # Population by sex over time

### Voter Data

With the voter data we can use pivot tables in more creative an interesting ways to get summaries.

Perhaps the first trick is to transform the column `VotedFor` into what are known as dummy variables; they take value 1 when a condition holds, and 0 when it does not hold. In this case, we get a dummy variable for when a voter voted for Clinton, Johnson, Stein, or Trump, which can be more useful than a single column containing which candidate someone voted for.

We can construct a `DataFrame` containing dummy variables from a `Categorical` `Series` using the pandas function `get_dummies()`.

In [ ]:
pd.get_dummies(voters.VotedFor).head()

We then join these tables together. Notice the relationship between the dummy variables and the `VotedFor` column.

In [ ]:
voters = voters.join(pd.get_dummies(voters.VotedFor))
voters.head()

In [ ]:
voters["Third Party (LG)"] = voters["Stein (G)"] + voters["Johnson (L)"]    # An intersting column itself
voters

When we have dummy variables, if we aggregate using `'sum'`, we get the number of voters in a cell who voted for a certain candidate.

In [ ]:
voters.pivot_table(values="Clinton (D)", index="Education", columns="AgeGroup", aggfunc="sum")

Replace `"sum"` with `"mean"`, though, and you interpret the number in the cell as the *proportion* of individuals in that group who chose a certain candidate. This allows you to see which groups favored which candidate and by how much.

In [ ]:
voters.pivot_table(values="Clinton (D)", index="Education", columns="AgeGroup", aggfunc='mean')

The above table suggests that in our sample Clinton was more popular among older voters and among more educated voters.

In [ ]:
voters.pivot_table(values="Trump (R)", index="Education", columns="AgeGroup", aggfunc='mean')    # Agrees with pattern

In [ ]:
voters.pivot_table(values="Registered", index="Education", columns="AgeGroup", aggfunc='sum')    # How many registered?

In [ ]:
voters.pivot_table(values="Registered", index="Education", columns="AgeGroup", aggfunc='mean')    # What proportion?